In [4]:
import sys
import torch
from molfeat.trans.pretrained import PretrainedDGLTransformer

sys.path.append('..')

from src.tacogfn.models.beta_docking_score_predictor import DockingScorePredictionModel
from src.tacogfn.data.pharmacophore import PharmacoDB

/home/tsa87/anaconda3/envs/tacogfn/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
model_state = torch.load('../logs/debug_docking_score_prediction_beta/model_state_23.pt')

In [6]:
model = DockingScorePredictionModel(hidden_dim=model_state['hps']['hidden_dim'])
model.load_state_dict(model_state['models_state_dict'][0])

<All keys matched successfully>

In [7]:
db = PharmacoDB('../misc/pharmacophores_db.lmdb/')

In [8]:
pdbs = [
    '4u5s_A',
    '2z3h_A',
    '3dzh_A',
]
smis = [
    'CNC(=O)c1cc(Cl)ccn1',
    'CN(CCC(N)CC(=O)NC1CCC(N2C=CC(N)(O)NC2=O)OC1C(=O)O)C(=N)N',
    'NC1=c2ncn3c2=NCN1C1CC(COP(=O)(O)OP(=O)(O)OCC2OC3C(O)C2O)C(O)C1O',
]

In [9]:
transformer = PretrainedDGLTransformer(kind='gin_supervised_contextpred', dtype=float)

In [10]:
datas = [db.get_data_from_id(pdb) for pdb in pdbs]

In [11]:
ligand_features = torch.tensor(transformer(smis))

In [12]:
from torch_geometric.data import DataLoader

loader = DataLoader([{
    'pharmacophore': p, 
    'ligand_features': l
} for p, l in zip(datas, ligand_features)], batch_size=1000)
for i in loader:
    print(len(model(i)))

3


/home/tsa87/anaconda3/envs/tacogfn/lib/python3.9/site-packages/torch_geometric/deprecation.py:22: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)
